In [7]:
#Implementation of the protocol from https://arxiv.org/pdf/2007.13126.pdf
#Honest verifier and fcg
#Advantage for the honest verifier and fcg setting : generator testing completely suffices

%matplotlib inline
import numpy as np
# Importing standard Qiskit libraries and configuring account
import qiskit
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import math as math
# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import depolarizing_error
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

simulator2 = Aer.get_backend('statevector_simulator')

#select graph structure by giving the adjacency matrix

adj_matrix = [[0,1,0],[1,0,1],[0,1,0]]

def Convert(string): 
    list1=[] 
    list1[:0]=string 
    return list1

def stabilizer_test(circuit,num_qubits,alpha):
    for i in range(0,num_qubits):
        if i == alpha:
            circuit.h(i)
            circuit.measure(i,i)
        if adj_matrix[alpha][i] == 0:
            pass
        if adj_matrix[alpha][i] == 1:
            circuit.measure(i,i)
    return(circuit)

def stabilizer_test_with_depolarizing_noise_line(noise_param,J,m):
    #countslist = [] #hilfsvariable um die ergebnisse zu checken
    n_pass = 0
    num_tests = math.ceil(m*J**4*math.log(J))
    num_qubits = J
    for alpha in range(J):
        for beta in range(num_tests):
            ### white noise model/depolarizing channel
            graph_state_circuit = qiskit.circuit.library.GraphState(adj_matrix)
            new_creg = graph_state_circuit._create_creg(len(graph_state_circuit.qubits), 'meas')
            graph_state_circuit.add_register(new_creg)
            graph_state_circuit.barrier()
            qiskit.providers.aer.noise.depolarizing_error(noise_param,num_qubits)
            noise_model = NoiseModel()
            error = depolarizing_error(noise_param,num_qubits=1) #1qb noise
            error2 = depolarizing_error(noise_param,num_qubits=2)#2qb noise
            noise_model.add_all_qubit_quantum_error(error,['h'])
            noise_model.add_all_qubit_quantum_error(error2,['cz'])
            basis_gates = noise_model.basis_gates
            graph_state_circuit = stabilizer_test(graph_state_circuit,num_qubits,alpha)
            simulator = Aer.get_backend('qasm_simulator')
            shots = 1
            result = execute(graph_state_circuit,simulator,basis_gates=basis_gates,shots=shots,noise_model=noise_model).result()
            counts = result.get_counts(graph_state_circuit)
            #if product over all returns equals 1, accept
            dummy = list(counts.values())
            j = 0
            #countslist.append(counts)
            for i in list(counts):
                string_i = Convert(i) #represent element as list of strings to iterate over returned element 0000 = ['0','0','0','0']
                brauchbarer_string_i = [int(x) for x in string_i]
                if np.sum(brauchbarer_string_i) % 2 == 0:
                    n_pass += int(dummy[j])
                j += 1
    return n_pass/(num_tests*J)



ibmqfactory.load_account:WARNING:2022-02-08 14:58:57,547: Credentials are already in use. The existing account in the session will be replaced.


In [12]:
noise_parameter = [0.05 for i in range(20)]

percentage_passed = [stabilizer_test_with_depolarizing_noise_line(noise_param,3,23) for noise_param in noise_parameter]

print(percentage_passed)
print(np.mean(percentage_passed),np.std(percentage_passed))

[0.9099495196222114, 0.9135319980459209, 0.9182543559680834, 0.9091353199804592, 0.9140205178309722, 0.9140205178309722, 0.9146718775443738, 0.9053900016283993, 0.9211854746783912, 0.9086468001954079, 0.9086468001954079, 0.9241165933886989, 0.9158117570428269, 0.9149975574010747, 0.9163002768278782, 0.9138576779026217, 0.9135319980459209, 0.9146718775443738, 0.9151603973294252, 0.91320631818922]
0.9139553818596321 0.004221733099109159
